# Opportunity Example - NER Module

The NER Module offers a comprehensive set of tools and utilities for Named Entity Recognition tasks. The accompanying Read-the-Docs documentation provides step-by-step guidance on how to effectively use the module’s various functions.

**Recommendation**: We strongly suggest using an OpenAI API key, as the agents are optimized for the OpenAI infrastructure. This ensures the best performance and compatibility.

This notebook will walk you through the proper implementation and usage of the module for your specific use case.

First of all, create a new conda/venv environment with these commands

In [22]:
# For conda environment
# !conda create -n nerenv python=3.10
# !conda activate nerenv
# For venv environment
# !python -m venv nerenv
# Linux/macOS case
# !source nerenv/bin/activate
# Activate it (Windows)
# !.\nerenv\Scripts\activate

And inside the new environment, install the dependencies

In [23]:
# !pip install -r requirements.txt

**IMPORTANT**: Remember to activate the environment before starting

In [24]:
import requests
import os
import pandas as pd
import logging
from pydantic import BaseModel, Field, model_validator
from dotenv import load_dotenv
from datetime import datetime, timedelta
from typing import Optional
from ner_module import NER, ToolsManager
from rapidfuzz import fuzz, process

This command is used only to suppress the urllib3 warnings.

In [25]:
logging.getLogger("urllib3.connection").setLevel(logging.ERROR)  # Suppress urllib3 warnings

Do not hard-code your API KEY, create a .env file and use ```load_dotenv``` in your script.

In [26]:
load_dotenv() # Returns True if the .env file was loaded successfully, False otherwise

True

In [27]:
API_ID = os.getenv("API_ID")
API_PWD = os.getenv("API_PWD")
OPENAI_API = os.getenv("OPENAI_API")

You provided an Excel file containing the client names.

Use the ```pandas``` library to extract and process the data.

In [28]:
CLIENTS = pd.read_excel("lista_clienti.xlsx")
CLIENTS

,ancl_id,ancl_codice,ancl_ragione_sociale,ancl_indirizzo,ancl_pariva,ancl_codfisc
0,1,16206,CLIENTE VINCHIATURO 16206,CONTRADA LE MACERE SNC,1724220700,NaN
1,2,16207,CLIENTE APRILIA 16207,VIA DELL'INDUSTRIA 10/12,2999410596,2999410596
2,3,16208,CLIENTE OSPEDALETTO D'ISTRANA 16208,"VIA DELLE FORNACI, 14",3819031208,792100265
3,4,1621,CLIENTE CALENZANO 1621,VIA SALVANTI N. 81,3871570481,3871570481
4,5,16212,CLIENTE PALAZZO DI ASSISI 16212,"VIA DELLE MACINE, 8",NaN,BNCLLN48C41G478J
...,...,...,...,...,...,...
995,996,16040,CLIENTE CISTERNA DI LATINA 16040,"VIA APPIA KM 56,700",1296411000,NaN
996,997,16042,CLIENTE PERUGIA 16042,"VIA G.B. PONTANI, 10",3040410544,NaN
997,998,16044,CLIENTE PRATO 16044,"VIA RIMINI, 7",1727040972,NaN
998,999,16055,CLIENTE MONTECASTRILLI 16055,"LOC.PALOMBARO, 5",1433320551,NaN


From here you can retrieve the client names and store them.

In [29]:
CLIENTS_LIST = [client.strip().lower() for client in CLIENTS["ancl_ragione_sociale"].to_list()]
CLIENTS_LIST

['cliente vinchiaturo 16206',
 'cliente aprilia 16207',
 "cliente ospedaletto d'istrana 16208",
 'cliente calenzano 1621',
 'cliente palazzo di assisi 16212',
 "cliente porto sant'elpidio 16222",
 'cliente serravalle pistoiese 1061',
 'cliente porto valtravaglia 1079',
 'cliente bellusco 1080',
 'cliente biccari 1085',
 'cliente crespellano 1111',
 'cliente solicchiata-castiglione di sic 1112',
 'cliente castel goffredo 11619',
 'cliente castelnuovo garfagnana 1059',
 'cliente sesto fiorentino 1090',
 'cliente campoligure 1106',
 'cliente empoli 1115',
 'cliente fiumana 11622',
 'cliente correggio 11629',
 'cliente pisticci scalo 16209',
 'cliente spoltore 16210',
 'cliente pozzilli 16211',
 'cliente palazzo di assisi 16213',
 'cliente alessandria 16223',
 'cliente desenzano del garda 16226',
 'cliente battipaglia 11631',
 'cliente cerrina monferrato 16224',
 'cliente la morra 16227',
 "cliente canonica d'adda 16243",
 'cliente torino 16247',
 'cliente siziano 16248',
 'cliente carate 

Before use your APIs, you have to retrieve the token.

In [ ]:
HOST = "https://opportunitystatwolf.opportunitycrm.it/API"

response = requests.post(
    f"{HOST}/auth/token",
    json={
        "apiId": API_ID,
        "apiPassword": API_PWD
    }
)

print(response.status_code)

if response.status_code == 200:
    print("Authentication successful. Token received.")
    data = response.json()
    token = data.get("token")
    refresh_token = data.get("refreshToken")
else:
    print(response.status_code)

HEADERS = {
    "authorization": f"Bearer {token}"
}

200
Authentication successful. Token received.


This is an example of how to call the APIs, this function returns the activities that the NER module will use to recognize the specific activity.

In [31]:
def get_activity_list() -> list[str]:
    response = requests.get(
        f"{HOST}/attivita/tipi",
        headers=HEADERS
    )
    activity_list = []

    if response.status_code == 200:
        print("Activity list retrieved successfully.")
        activities = response.json()
        for activity in activities:
            id = activity.get("id", "")
            codice = activity.get("codice", "")
            descrizione = activity.get("descrizione", "")
            todo = activity.get("todo", False)
            act = f"id: {id}, codice: {codice}, descrizione: {descrizione}, todo: {todo}"
            activity_list.append(act)
    else:
        print("Failed to retrieve activity list.")

    return activity_list

activity_list = get_activity_list()
activity_list

Activity list retrieved successfully.


['id: 2, codice: CH_TODO, descrizione: Chiusura ToDo, todo: False',
 'id: 5, codice: todo, descrizione: Generico ToDo, todo: True',
 'id: 6, codice: incontro_cliente, descrizione: Esito Incontro, todo: False',
 'id: 7, codice: FIS_INC, descrizione: Pianifica Incontro, todo: True',
 'id: 12, codice: OFF_OK, descrizione: Off. Da Spedire, todo: True',
 'id: 13, codice: CH_OFF_OK, descrizione: Off. Spedita, todo: False',
 'id: 14, codice: CH_LAM_3, descrizione: Chiusura Lamentela (terzi), todo: False',
 'id: 15, codice: CH_IN_DOC_3, descrizione: Documentazione inviata al Cliente (terzi), todo: False',
 'id: 16, codice: CH_RIC_CLI, descrizione: Telefonata Cliente Effettuata, todo: False',
 'id: 17, codice: CH_TD_GEN_3, descrizione: Chiusura ToDo Generico (terzi), todo: False',
 'id: 18, codice: CH_TEL_OFF, descrizione: Telefonata Scad. Off, todo: False',
 'id: 19, codice: CH_CR_OFF, descrizione: Offerta Creata, todo: False',
 'id: 20, codice: CH_CR_OFF_3, descrizione: Offerta Creata (terzi)

**ONLY FOR TESTING PURPOSES**: This function is to get the ```user_id```, it should be retrieved from the metadata.

In [32]:
def get_user_id() -> int:
    return 38 # ONLY FOR TESTING PURPOSES -> THIS ID SHOULD BE DYNAMICALLY RETRIEVED FROM THE METADATA

get_user_id()

38

The model now requires a structured output, which means you must clearly specify the parameters it should recognize during the entity extraction process. To achieve this, create a subclass of ```BaseModel```, as shown below. Make sure to provide clear and detailed descriptions for each parameter, so the model can handle every field correctly.

The ```@model_validatior``` are also available. During the recognition, there will be a validation step, a validation step will run to check whether any parameters are missing or incorrectly filled. These validator functions can be used to adjust or populate parameters as needed. For example, if you want certain activities to have their dates assigned automatically, this should be done before validation.

In [33]:
class Activity(BaseModel):
    id_attivita: int = Field(..., description="ID dell'attività individuata.")
    is_todo: bool = Field(..., description="Indica se l'attività è un todo (da fare) o meno. E' strettamente collegato all'attività. Se True indica un'attività in corso d'opera, False un'attività completata.")
    codice_todotype: Optional[str] = Field(default=None, description="Codice del tipo di attività, presente vicino alla descrizione dell'attività.")
    id_stato: int = Field(..., description="ID dello stato dell'attività. 1 = Aperto, 2 = Chiuso, 3 = In corso.")
    data_registrazione: datetime = Field(default=datetime.now(), description="Data di registrazione dell'attività in formato ISO 8601, che corrisponde alla data attuale.")
    data_inizio: datetime = Field(..., description="Data di inizio dell'attività in formato ISO 8601 che l'utente deve definire.")
    data_fine: datetime = Field(..., description="Data di fine dell'attività in formato ISO 8601 che l'utente deve definire.")
    data_esecuzione: Optional[datetime] = Field(default=None, description="Data di esecuzione dell'attività in formato ISO 8601. Se non specificata, viene impostata a None, viene cambiata quando viene effettivamente eseguito il todo.")
    giornata_intera: bool = Field(default=False, description="Indica se l'attività è di tutta la giornata.")
    oggetto: Optional[str] = Field(default="", description="Oggetto dell'attività.")
    testo: Optional[str] = Field(default="", description="Testo dell'attività. Completa descrizione delle informazioni rilevate nel testo.")
    note: Optional[str] = Field(default="", description="Note aggiuntive sull'attività. Note aggiuntive utili, come date, luoghi o altro.")
    destinazione: Optional[str] = Field(default=None, description="Destinazione (indirizzo, via, città) specificato per l'attività.")
    id_utente: int = Field(default=get_user_id(), description="ID dell'utente che ha creato l'attività.")
    id_calendario: Optional[int] = Field(default=None, description="ID del calendario dell'utente dove inserire l'attività.")
    cliente: str = Field(default="", description="Cliente associato all'attività")
    id_cliente: Optional[int] = Field(default=None, description="ID del cliente associato al contatto. Azienda in cui lavora il contatto se presente.")
    contatto: Optional[str] = Field(default=None, description="Nome completo del contatto associato all'attività.")
    id_contatto: Optional[int] = Field(default=None, description="ID del contatto associato all'attività.")
    id_att_parent: Optional[int] = Field(default=None, description="ID dell'attività padre, se presente. Utilizzato per attività ricorrenti o correlate.")

    @model_validator(mode="before")
    def set_some_data(cls, values):
        print("Before model validation.")
        """
        if values.get("id_attivita") in [1, 2, 3]:
            if values.get("data_registrazione") is None:
                values["data_registrazione"] = datetime.now()
            if values.get("data_inizio") is None:
                values["data_inizio"] = values["data_registrazione"] + timedelta(days=1) # Default to one day after registration date if not specified
            if values.get("data_fine") is None or values.get("data_inizio") >= values.get("data_fine"):
                values["data_fine"] = values["data_inizio"] + timedelta(hours=1) # Default to one hour after start date if not specified
        """
        return values

    @model_validator(mode="after")
    def set_data_fine_if_missing(self):
        print("After model validation.")
        return self

This is the text extracted from the vocal note sent by the user.

In [34]:
text = "Dopo un confronto con Siziano siamo pronti a procedere con l'invio dell'offerta."
print("\n\nINPUT TEXT\n", text, "\n")



INPUT TEXT
 Dopo un confronto con Siziano siamo pronti a procedere con l'invio dell'offerta. 



Create a ```ToolsManager``` instance to handle appropriately the tools that you will create.

> **Tools** are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models. [Tools LangChain](https://python.langchain.com/docs/integrations/tools/)

In [35]:
tools_manager = ToolsManager(host=HOST, bearer_token=token)

Create tools using the following format: the main function should contain a nested function that is returned as a ```Callable```.

**IMPORTANT**: Provide a detailed docstring for the nested function. This docstring must clearly explain:
- What the function does
- Its parameters
- Its return values

The LLM will read this description and decide whether or not to call the function based on the information provided.

In [ ]:
# Tool to retrieve the client ID from the client name
def make_get_client_id_tool(host: str, token: str):
    headers = {"Authorization": f"Bearer {token}"}
    def get_client_id(client_name):
        """
        Retrieve the client ID given a client name using fuzzy matching.

        :param client_name: Name of the client (partial or full)
        :return: The ID of the client if found, otherwise None.
        """
        THR = 50
        clients = pd.read_excel("lista_clienti.xlsx")
        clients_list = [client.strip().lower() for client in clients["ancl_ragione_sociale"].to_list()]

        client_name = client_name.strip().lower()
        match = process.extractOne(client_name, clients_list, scorer=fuzz.token_sort_ratio)
        best_match, score = match[0], match[1]
        client = best_match if score >= THR else None
        if client:
            params = {
                "RagSoc": client
            }
            response = requests.get(
                f"{host}/clienti",
                headers=headers,
                params=params
            )

            if response.status_code == 200:
                client = response.json()[0]
                return client.get("id", None)
            else:
                return None
        else:
            return None
        
    return get_client_id

def make_get_todo_from_activity(host: str, token: str):
    headers = {"Authorization": f"Bearer {token}"}
    def get_todo_from_activity(activity_id: int):
        """
        Get if the activity is a todo or not.

        :param activity_id: The ID of the activity.
        :return: True if the activity is a todo, False otherwise.
        """
        response = requests.get(
            f"{host}/attivita/tipi",
            headers=headers
        )

        if response.status_code == 200:
            data = response.json()
            for activity in data:
                if activity.get("id") == activity_id:
                    return activity.get("todo", False)
        
        return False
    
    return get_todo_from_activity

def make_get_calendar_id_tool(host: str, token: str):
    headers = {"Authorization": f"Bearer {token}"}
    def get_calendar_id(user_id, default = True):
        """
        Retrieve the calendar ID using the user_id.

        :param user_id: ID of the user
        :param default: If True, return the default calendar ID for the user; otherwise, return the secondary calendar ID.
        :return: The calendar ID for the user.
        """
        params = {"idUtente": user_id}
        response = requests.get(
            f"{host}/utenti/calendariUtente",
            headers=headers,
            params=params
        )
        user_calendars = response.json()
        for calendar in user_calendars:
            if calendar.get("default", False) == default:
                return calendar.get("idCalendario")

    return get_calendar_id
    
def make_get_todotype(host: str, token: str):
    headers = {"Authorization": f"Bearer {token}"}
    def get_todotype(id_activity: int):
        """
        Retrieve the TodoType using the activity ID.
        The TodoType is the code relative to the activity type.

        :param id_activity: The ID of the activity.
        :return: The TodoType if found, otherwise None.
        """
        response = requests.get(
            f"{host}/attivita/tipi",
            headers=headers
        )

        if response.status_code == 200:
            activities = response.json()
            for activity in activities:
                if activity.get("id") == id_activity:
                    return activity.get("codice", None)
            return None
        else:
            return None

    return get_todotype

def make_get_all_users(host: str, token: str):
    headers = {"Authorization": f"Bearer {token}"}
    def get_all_users():
        """
        Return the list of all the users.
        This function should be called ONLY when an activity is assigned to another person or entity (e.g. "assign this activity to John Doe/the marketing team").

        :return: A list of all users.
        """
        response = requests.get(
            f"{host}/utenti",
            headers=headers
        )
        if response.status_code == 200:
            users = response.json()
            users_infos = []
            for user in users:
                infos = f"ID: {user.get('id', '')}, Fullname: {user.get('fullname', '')}, Name: {user.get('nome', '')}, Cognome: {user.get('cognome', '')}, Email: {user.get('email', '')}, Cellphone: {user.get('cellulare', '')}"
                users_infos.append(infos)
            return users_infos
        else:
            return []

    return get_all_users

def make_get_current_date():
    def get_current_date():
        """
        Get the current day, month, year and time.
        
        :return: The current day, month, year and time as a formatted string.
        """
        return datetime.now().strftime("%A %d %B %Y %H %M %S")
    return get_current_date

Add the tools to the tool manager instance.

In [37]:
tools_manager.add_fn_to_tools(make_get_all_users(tools_manager.host, tools_manager.token))
tools_manager.add_fn_to_tools(make_get_client_id_tool(tools_manager.host, tools_manager.token))
tools_manager.add_fn_to_tools(make_get_calendar_id_tool(tools_manager.host, tools_manager.token))
tools_manager.add_fn_to_tools(make_get_todo_from_activity(tools_manager.host, tools_manager.token))
tools_manager.add_fn_to_tools(make_get_todotype(tools_manager.host, tools_manager.token))
tools_manager.add_fn_to_tools(make_get_current_date())

In [38]:
print(tools_manager)

ToolsManager with 6 tools: ['get_all_users', 'get_client_id', 'get_calendar_id', 'get_todo_from_activity', 'get_todotype', 'get_current_date']


You can create the NER instance now, pass your API KEY.

In [39]:
ner = NER(provider="openai", api_key=OPENAI_API)

If you are not sure about your key, you can check using the method ```health_check```.

In [40]:
ner.health_check()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'status': 'healthy', 'model_name': 'gpt-4o-mini', 'temperature': 0.0}

In the case there are multiple activities inside the text, you can check and divide in different tasks that will be send separetly to the LLM.

In [41]:
activities = ner.detect_multiactivities(text=text)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [42]:
print("Number of activities detected: ", len(activities))
for i, act in enumerate(activities, start=1):
    print(f"Activity {i}: {act}")

Number of activities detected:  1
Activity 1: Dopo un confronto con Siziano siamo pronti a procedere con l'invio dell'offerta.


In [43]:
example_text = "Ricordami domani di richiamare Alberto Rossi di PinkPalla e anche di inviare la documentazione all'Ufficio Tecnico."
example_activities = ner.detect_multiactivities(text=example_text)

print("Number of activities detected: ", len(example_activities))
for i, act in enumerate(example_activities, start=1):
    print(f"Activity {i}: {act}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Number of activities detected:  2
Activity 1: Ricordami domani di richiamare Alberto Rossi di PinkPalla
Activity 2: Ricordami domani di inviare la documentazione all'Ufficio Tecnico


You can now call the ```chat``` method to perform entity recognition.

- text: The input text from which the model should extract entities.
- tasks: In your case, this is the list of all activities. Providing the complete list allows the LLM to determine the relevant activity for each extracted entity.
- base_model: The ```BaseModel``` subclass you created earlier.
- tools_manager: The ```ToolsManager``` instance containing your custom tools.
- ask_missing_params_with_llm: Set to ```True``` if you want the model to generate *human-like* guesses for missing parameters; otherwise, set to ```False```.
- return_raw_result: Set to ```True``` if you want the response as a NERResult object; otherwise, the method will return a dict.

The additional parameters should be sent in this format:
```json
    {"parameter": "user_answer"}
```
**Example**:
It is possible that the NER module does not recognise some entities in the provided text, so it asks for them.
In your case, it could be the ```data_inizio``` and ```data_fine```.
The module will return:
```json
    {
        "success": False,
        "status": "missing_parameter",
        "questions": [{"parameter": "data_inizio", "question": "Per favore, inserisci la data di inizio."}, {"parameter": "data_fine", "question": "Per favore, inserisci la data di fine."}]
    }
```
Although you could handle these questions in different ways, however the user_input parameter should be formatted as follows:
```json
    {
        "data_inizio": "Domani mattina alle 10",
        "data_fine": "Domani mattina alle 11"
    }
```

The ```chat``` method will call the ```dynamic_type_casting``` method, that will handle the responses and generate the correct format.

- *Domani mattina alle 10 -> 2025-09-23T10:00:00*
- *Domani mattina alle 11 -> 2025-09-23T11:00:00*

See ```ner_module.utils``` for ```NERResult``` class, or check the *Read-the-docs* documentation. 
If you don't want to manage ```NERResult``` objects, leave *return_raw_result* to default (False) and work with dict.

In [44]:
import json
from ner_module.utils import NERResult

for text in activities:
    print("PROCESSING TEXT: ", text)
    print("-" * 80)
    complete = False
    additional_parameters = {}
    while not complete:
        result = ner.chat(
            text=text,
            tasks=activity_list,
            base_model=Activity,
            tools_manager=tools_manager,
            ask_missing_params_with_llm=True,
            user_inputs=additional_parameters if additional_parameters else None,
            return_raw_result=True
        )
        additional_parameters.clear()
        if isinstance(result, NERResult) and result.success:
            complete = result.success
        elif isinstance(result, NERResult) and not result.success and result.status == "missing_parameter":
            # The parameters can be requested in different ways to the user
            for item in result.questions:
                item = json.loads(item) if isinstance(item, str) else item
                param, question = item.values()
                answer = input(question)
                additional_parameters[param] = answer
        else:
            raise RuntimeError("NER processing failed.")

    print("-" * 80)
    print("FINAL RESULT: ")
    result_data = json.loads(result.data)
    for key, value in result_data.items():
        print(f"{key}: {value}" if value is not None else f"{key}: None")

PROCESSING TEXT:  Dopo un confronto con Siziano siamo pronti a procedere con l'invio dell'offerta.
--------------------------------------------------------------------------------


INFO:ner_module.llm_client:Determined processing mode: agent_with_tools
INFO:ner_module.llm_client:Initializing agent with 6 tools: ['get_all_users', 'get_client_id', 'get_calendar_id', 'get_todo_from_activity', 'get_todotype', 'get_current_date']




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:ner_module.tools:Calling tool function: get_client_id with args: (), kwargs: {'client_name': 'Siziano'}



Invoking: `get_client_id` with `{'client_name': 'Siziano'}`




...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
data_inizio
  Input should be a valid datetime or date, input is too short [type=datetime_from_date_parsing, input_value='None', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/datetime_from_date_parsing
data_fine
  Input should be a valid datetime or date, input is too short [type=datetime_from_date_parsing, input_value='None', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/datetime_from_date_parsing
INFO:ner_module.llm_client:Missing parameters identified: []


None

> Finished chain.
Before model validation.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:ner_module.llm_client:Missing parameters after tool recovery: []
INFO:ner_module.llm_client:Missing parameters identified: []
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:ner_module.llm_client:Missing parameters after tool recovery: []
ERROR:ner_module.llm_client:Validation error during structured output processing: 2 validation errors for Activity
data_inizio
  Input should be a valid datetime or date, input is too short [type=datetime_from_date_parsing, input_value='None', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/datetime_from_date_parsing
data_fine
  Input should be a valid datetime or date, input is too short [type=datetime_from_date_parsing, input_value='None', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/datetime_from_date_parsing


Before model validation.


RuntimeError: NER processing failed.